<a href="https://colab.research.google.com/github/dev-researcher/automatas/blob/main/Class_W8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class DFA:
    def __init__(self, Q, Sigma, delta, q0, F):
        self.Q = Q          # Set of states
        self.Sigma = Sigma  # Alphabet
        self.delta = delta  # Transition function
        self.q0 = q0        # Initial state
        self.F = F          # Set of final states

class NFA:
    def __init__(self, Q, Sigma, delta, q0, F):
        self.Q = Q          # Set of states
        self.Sigma = Sigma  # Alphabet
        self.delta = delta  # Transition function (now allows multiple next states)
        self.q0 = q0        # Initial state
        self.F = F          # Set of final states

    def epsilon_closure(self, states):
        """Compute epsilon closure for given set of states."""
        closure = set(states)
        stack = list(states)
        while stack:
            state = stack.pop()
            if 'ε' in self.delta.get(state, {}):
                for next_state in self.delta[state]['ε']:
                    if next_state not in closure:
                        closure.add(next_state)
                        stack.append(next_state)
        return closure

    def move(self, states, symbol):
        """Compute possible next states for given states and input symbol."""
        next_states = set()
        for state in states:
            if symbol in self.delta.get(state, {}):
                next_states.update(self.delta[state][symbol])
        return next_states

    def accepts(self, word):
        current_states = {self.q0}
        for symbol in word:
            next_states = set()
            for state in current_states:
                if symbol in self.delta.get(state, {}):
                    next_states.update(self.delta[state][symbol])
            current_states = next_states
        return bool(current_states.intersection(self.F))

def construct_phone_nfa(dfa, number):
    digit_to_letters = {
        '2': 'ABC', '3': 'DEF', '4': 'GHI', '5': 'JKL',
        '6': 'MNO', '7': 'PQRS', '8': 'TUV', '9': 'WXYZ'
    }

    Q = set(dfa.Q).union({f'q_{i}' for i in range(len(number)+1)})
    Sigma = set(dfa.Sigma)
    delta = {state: {} for state in Q}
    q0 = f'q_0'
    F = set()

    # Add transitions for each digit
    for i, digit in enumerate(number):
        current_state = f'q_{i}'
        next_state = f'q_{i+1}'

        for letter in digit_to_letters[digit]:
            if letter not in delta[current_state]:
                delta[current_state][letter] = set()
            delta[current_state][letter].add(next_state)

            # Add transitions from DFA
            for dfa_state in dfa.Q:
                if letter in dfa.delta.get(dfa_state, {}):
                    dfa_next_state = dfa.delta[dfa_state][letter]
                    if letter not in delta[next_state]:
                        delta[next_state][letter] = set()
                    delta[next_state][letter].add(dfa_next_state)

    # Set final states
    for dfa_final_state in dfa.F:
        F.add(dfa_final_state)

    return NFA(Q, Sigma, delta, q0, F)



In [ ]:
# Example usage
# Create a simple DFA for a few English words
Q = {'q0', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6'}
Sigma = set('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
delta = {
    'q0': {'A': 'q1', 'B': 'q3', 'C': 'q5'},
    'q1': {'D': 'q2', 'E': 'q2'},
    'q2': {'D': 'q2'},
    'q3': {'E': 'q4'},
    'q4': {'D': 'q2', 'E': 'q2'},
    'q5': {'A': 'q6'},
    'q6': {'T': 'q2'}
}
q0 = 'q0'
F = {'q2'}

english_dfa = DFA(Q, Sigma, delta, q0, F)

phone_number = "233"
phone_nfa = construct_phone_nfa(english_dfa, phone_number)

# Test some words
test_words = ["ADD", "BED", "BEE", "CAT", "DOG"]
for word in test_words:
    if phone_nfa.accepts(word):
        print(f"'{word}' is a valid word for the number {phone_number}")
    else:
        print(f"'{word}' is not a valid word for the number {phone_number}")

'ADD' is a valid word for the number 233
'BED' is a valid word for the number 233
'BEE' is a valid word for the number 233
'CAT' is not a valid word for the number 233
'DOG' is not a valid word for the number 233


In [ ]:
class CollatzTransducer:
    def __init__(self):
        self.states = {1, 2, 3, 4, 5, 6}
        self.initial_state = 1
        self.final_states = {2, 6}
        self.transitions = {
            1: {(0, 0): 2, (1, 0): 4, (0, 1): 3},
            2: {(0, 0): 2, (1, 1): 3},
            3: {(0, 1): 3, (1, 1): 3},
            4: {(0, 0): 5, (1, 1): 4},
            5: {(0, 1): 6, (1, 0): 4},
            6: {(0, 0): 6}
        }

    def process(self, input_bits, output_bits):
        current_state = self.initial_state
        for in_bit, out_bit in zip(input_bits, output_bits):
            if (in_bit, out_bit) not in self.transitions[current_state]:
                return False
            current_state = self.transitions[current_state][(in_bit, out_bit)]
        return current_state in self.final_states

    def collatz_function(self, n):
        if n % 2 == 0:
            return n // 2
        else:
            return 3 * n + 1

    def to_binary(self, n):
        return [int(b) for b in bin(n)[2:][::-1]]  # Reverse to get least significant bit first

    def pad_binary(self, bits, length):
        return bits + [0] * (length - len(bits))

    def verify_pair(self, n):
        collatz_n = self.collatz_function(n)
        n_bits = self.to_binary(n)
        collatz_n_bits = self.to_binary(collatz_n)

        max_length = max(len(n_bits), len(collatz_n_bits))
        padded_n = self.pad_binary(n_bits, max_length)
        padded_collatz_n = self.pad_binary(collatz_n_bits, max_length)

        return self.process(padded_n, padded_collatz_n)

# Test the transducer
collatz_transducer = CollatzTransducer()

test_numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
for n in test_numbers:
    result = collatz_transducer.verify_pair(n)
    print(f"n = {n}, f(n) = {collatz_transducer.collatz_function(n)}, Accepted: {result}")

n = 1, f(n) = 4, Accepted: True
n = 2, f(n) = 1, Accepted: False
n = 3, f(n) = 10, Accepted: True
n = 4, f(n) = 2, Accepted: False
n = 5, f(n) = 16, Accepted: True
n = 6, f(n) = 3, Accepted: False
n = 7, f(n) = 22, Accepted: True
n = 8, f(n) = 4, Accepted: False
n = 9, f(n) = 28, Accepted: False
n = 10, f(n) = 5, Accepted: False
